In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
%matplotlib inline
pylab.rcParams[ 'figure.figsize' ] = 10 , 8
from sklearn import preprocessing

In [2]:
Hotel_Reviews=pd.read_csv("Hotel_Reviews-2.csv")
Hotel_City_Reviews=pd.read_csv("Hotel_Info_Country_City.csv", usecols=['Hotel_Name','City'])
Hotel_City_Reviews.head()

,Hotel_Name,City
0,Hotel Arena,Amsterdam
1,K K Hotel George,London
2,Apex Temple Court Hotel,London
3,The Park Grand London Paddington,London
4,Monhotel Lounge SPA,Paris


In [58]:
Milan=Hotel_City_Reviews.loc[Hotel_City_Reviews['City']=='Milan']
Milan.Hotel_Name.unique()

array(['Hotel VIU Milan', 'Maison Borella',
       'Worldhotel Cristoforo Colombo', 'IH Hotels Milano Puccini',
       'Best Western Hotel City', 'Ch teau Monfort Relais Ch teaux',
       'Palazzo Parigi Hotel Grand Spa Milano', 'Hotel Lloyd',
       'Hotel Romana Residence', 'Hotel Crivi s', 'Hotel Galileo',
       'Carlyle Brera Hotel', 'Boscolo Milano Autograph Collection',
       'Mokinba Hotels King', 'Qualys Hotel Nasco',
       'Enterprise Hotel Design Boutique', 'IH Hotels Milano Ambasciatori',
       'NH Collection Milano President', 'Hotel Dieci', 'ME Milan Il Duca',
       'Hotel Principe Di Savoia', 'The Westin Palace', 'Glam Milano',
       'Excelsior Hotel Gallia Luxury Collection Hotel',
       'Rosa Grand Milano Starhotels Collezione', 'TownHouse 12',
       'Hotel Mozart', 'Hotel Dei Cavalieri',
       'Best Western Plus Hotel Galles', 'Starhotels Anderson',
       'Hotel Michelangelo', 'Hotel Mercure Milano Centro', 'NYX Milan',
       'The Yard Milano', 'Hotel Sunflo

In [59]:
Hotel_Milan = Hotel_Reviews[Hotel_Reviews['Hotel_Name'].isin(Milan.Hotel_Name.unique())]
len(Hotel_Milan)

38081

In [60]:
import langid
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/LiverJoker/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [61]:
corpusP= []
corpusN= []
uniqueUsefulTokens= set()

for i in Hotel_Milan.index:
       reviewP=""
       reviewN="" 
       if ("No Negative" in Hotel_Milan['Negative_Review'][i]) and "No Positive" in Hotel_Reviews['Positive_Review'][i]:
            continue
       elif "No Negative" in Hotel_Milan['Negative_Review'][i]:
            reviewP = Hotel_Milan['Positive_Review'][i]
       else:
            reviewN = Hotel_Milan['Negative_Review'][i]
       
       if (langid.classify(reviewP)[0]!='en' and langid.classify(reviewN)[0]!='en'):
            continue
       reviewP= re.sub('[^a-zA-Z]', ' ', reviewP)
       reviewN= re.sub('[^a-zA-Z]', ' ', reviewN)

       reviewP=reviewP.lower().split()
       reviewN=reviewN.lower().split()

       ps=PorterStemmer()
       reviewP  = [word for word in reviewP if not word in set(stopwords.words('english'))]
       reviewN  = [word for word in reviewN if not word in set(stopwords.words('english'))]
       for word in reviewP:
            uniqueUsefulTokens.add(word)
       reviewP = ' '.join(reviewP)
    
       for word in reviewN:
            uniqueUsefulTokens.add(word)
       reviewN = ' '.join(reviewN)
    
       if reviewP!="":
            corpusP.append(reviewP)
        
    
       if reviewN!="":
            corpusN.append(reviewN)
            
    

In [62]:
len(corpusP)

10181

In [63]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

def computeIDF(docList):
    import math
    ifdDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(),0)
    for doc in docList:
        for word, val in doc.items():
            if val>0:
                idfDict[word] +=1
            
    for word, val in idfDict.items():
        idfDict[word] = math.log(N/float(val))
        
    return idfDict

def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [64]:
from functools import reduce

In [65]:
bowA = []
for line in corpusP:
    line= re.sub('[^a-zA-Z]', ' ', line)
    bowA.append(line.split(" "))

bowAnew = reduce(lambda x,y: x+y,bowA)
len(bowA)

10181

In [66]:
wordSet=set(bowAnew)


In [67]:
wordDictAarr = np.array([])
for listbowA in bowA:
    wordDictA = dict.fromkeys(wordSet,0)
    for word in listbowA:
        wordDictA[word]+=1
    wordDictAarr = np.append(wordDictAarr, wordDictA)


In [68]:
i = 0
tfBowAarr = np.array([])
for listbowA in bowA:
    tfBowA = computeTF(wordDictAarr[i], listbowA)
    i +=1
    tfBowAarr = np.append(tfBowAarr, tfBowA)


In [69]:
idfs = computeIDF(wordDictAarr)
idfsData = pd.DataFrame.from_dict(idfs, orient='index', dtype=float)
idfsData[0]=1/idfsData[0]

PosRev = pd.DataFrame(idfsData[0].sort_values(ascending=False))
PositiveReviews = PosRev.iloc[0:10, :]
PositiveReviews

,0
staff,1.051391
location,1.015775
breakfast,0.842555
hotel,0.744004
good,0.679309
room,0.672610
great,0.655621
friendly,0.553922
helpful,0.553006
clean,0.526360


In [70]:
tfidfBowA = np.array([])
for tfBowA in tfBowAarr:
    tfidf = computeTFIDF(tfBowA, idfs)
    tfidf = {x:y for x,y in tfidf.items() if y!=0}
    tfidfBowA = np.append(tfidfBowA, tfidf)


#sum(tfidfBowA[1].values())
tfidfBowA[0]

{'appealing': 0.30482611916240177,
 'bathroom': 0.12345847835632785,
 'design': 0.13777287460641188,
 'dog': 0.48788451745063033,
 'first': 0.16269463152304606,
 'general': 0.20693897545078746,
 'good': 0.05257439904139564,
 'helpful': 0.06458209262304859,
 'highly': 0.15492604960305148,
 'hotel': 0.09600561310222547,
 'hotels': 0.15889839371413092,
 'little': 0.15629458047098496,
 'located': 0.1339933916134173,
 'nice': 0.06815762811779377,
 'nicest': 0.2655899659956836,
 'one': 0.12937415895754278,
 'outstanding': 0.182183719002218,
 'owners': 0.2800708627138323,
 'recommendable': 0.2903452224442531,
 'recommended': 0.17176563961823563,
 'rooms': 0.08562158151646486,
 'spacious': 0.23952484187536893,
 'staff': 0.03396859802412094,
 'super': 0.1325294864515174,
 'travelled': 0.2600845845732815}

In [71]:
OneHotList=[]
for i in range(0, len(tfidfBowA), 1):
    tempList=[]
    for key in tfidfBowA[i]:
        if tfidfBowA[i][key]>.05:
               tempList.append(key)
    OneHotList.append(tempList)

In [72]:
import collections 
import mlxtend

In [73]:
from mlxtend.preprocessing import OnehotTransactions
from mlxtend.frequent_patterns import apriori

In [74]:
oht = OnehotTransactions()
oht_ary = oht.fit(OneHotList).transform(OneHotList)  
df = pd.DataFrame(oht_ary, columns=oht.columns_)  
df

,aah,abandoned,abbondant,abilities,ability,abit,able,abnormally,abroad,absolute,...,z,zamero,zara,zeer,zone,zones,zoon,ztaff,ztl,zuppa
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
frequent_itemsets = apriori(df, min_support=0.05, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.051174,[bed],1
1,0.255967,[breakfast],1
2,0.139967,[clean],1
3,0.084962,[close],1
4,0.113250,[comfortable],1
5,0.078381,[everything],1
6,0.127198,[excellent],1
7,0.155191,[friendly],1
8,0.213535,[good],1
9,0.202829,[great],1
